In [7]:
import requests
import base64
import datetime
from urllib.parse import urlencode

In [2]:
client_id = 'cc734c51aec34525ace44bf1ba41b538'
client_secret = '087993d745b44fa7880fb5c922777a0a'

I'm pasting those `client id` directly from the dev website, BUT... it would be better to retrieve them using a scrypt, so you don't have to keep'em static and visible in our notebook.

In [3]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expires = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_id == None or client_secret == None:
            raise Exception('You must set client_id and client_secret')
        
        client_credentials = f'{client_id}:{client_secret}'
        client_credentials_b64 = base64.b64encode(client_credentials.encode())
        
        return client_credentials_b64.decode()

    def get_token_headers(self):
        client_credentials_b64 = self.get_client_credentials()
        
        return {
            'authorization': f'Basic {client_credentials_b64}'
        }

    def get_token_data(self):
        return {
            'grant_type': 'client_credentials'
        }

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()

        r = requests.post(token_url, data=token_data, headers=token_headers)

        if r.status_code not in range(200, 299):
            return False

        data = r.json()
        access_token = data['access_token']
        now = datetime.datetime.now()
        expires_in = data['expires_in']  # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expires = expires < now
        
        return True
        

In [4]:
spotify = SpotifyAPI(client_id, client_secret)

In [5]:
spotify.perform_auth()

True

In [6]:
access_token = spotify.access_token

Now that we have our `access_token` we can actually use it to make a call to the API.

In [10]:
headers = {
    'Authorization': f'Bearer {access_token}'
}
endpoint = 'https://api.spotify.com/v1/search'
data = urlencode({
    'q': 'Time', 
    'type': 'track'
})

look_up_url = f'{endpoint}?{data}'
print(look_up_url)

r = requests.get(look_up_url, headers=headers)
print(r.status_code)

https://api.spotify.com/v1/search?q=Time&type=track
200


In [11]:
r.json()

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Time&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3tJoFztHeIJkJWMrx0td2f'},
       'href': 'https://api.spotify.com/v1/artists/3tJoFztHeIJkJWMrx0td2f',
       'id': '3tJoFztHeIJkJWMrx0td2f',
       'name': 'Moneybagg Yo',
       'type': 'artist',
       'uri': 'spotify:artist:3tJoFztHeIJkJWMrx0td2f'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ'